# Derivation

Discretely many importance weights and rewards, control variates $\vec{\tau}$, maximum likelihood of sample $\{ (w_n, r_n, \vec{\tau}_n) \}_{n=1}^N$ from $h$ is 
\begin{alignat}{2}
&\!\max_{Q \succeq 0} &\qquad& \sum_n \log(Q_{w_n, r_n, \vec{\tau}_n}),\label{eq:mle} \\
&\text{subject to} &  & \sum_{w,r,\vec{\tau}} Q_{w,r,\vec{\tau}} = 1, \tag{$\beta$} \label{eq:mlesumw} \\
&                  &  & \sum_{w,r,\vec{\tau}} w Q_{w,r,\vec{\tau}} = 1, \tag{$\gamma$} \label{eq:mlesum} \\
&                  &  & \sum_{w,r,\vec{\tau}} \vec{\tau} Q_{w,r,\vec{\tau}} = 0, \tag{$\vec{\delta}$} \label{eq:mlesumdr}
\end{alignat}
Estimate is $\hat V(\pi) = \sum_{w,r,\vec{\tau}} Q_{w,r,\vec{\tau}} w r$.

Dual (ignoring constants) is $$
\ldots
$$

# Tests

## Censorship changes results

We learned this the hard way.

In [ ]:
data, wmin, wmax, censored = None, None, None, None
for data, wmin, wmax, censored in [
    # some data where exogenous censorship is discarded
   ([ (c, w, r) for c, w, r in [ 
      (86, -59.999996, 0.0), (44884, -1.0497237, 0.0), (16331, -1.0447762, 0.0), (31257, -1.0344828, 0.0), 
      (15868, -1.0, 0.0), (41332, 0.0, 0.0), (1958, 0.0, 1.0), (17763, 1.0, 0.0), (1339, 1.0, 1.0), 
      (30726, 1.0344828, 0.0), (3867, 1.0344828, 1.0), (2034, 1.0447762, 1.0), (16728, 1.0447762, 0.0), 
      (40629, 1.0497237, 0.0), (3445, 1.0497237, 1.0), (85, 59.999996, 0.0), (6, 59.999996, 1.0), 
     ]
     if w >= 0
    ], 0, 380, False),
    # same data where exogenous censorship is modeled
   ([ (c, -w if w < 0 else w, None if w < 0 else r) for c, w, r in [ 
      (86, -59.999996, 0.0), (44884, -1.0497237, 0.0), (16331, -1.0447762, 0.0), (31257, -1.0344828, 0.0), 
      (15868, -1.0, 0.0), (41332, 0.0, 0.0), (1958, 0.0, 1.0), (17763, 1.0, 0.0), (1339, 1.0, 1.0), 
      (30726, 1.0344828, 0.0), (3867, 1.0344828, 1.0), (2034, 1.0447762, 1.0), (16728, 1.0447762, 0.0), 
      (40629, 1.0497237, 0.0), (3445, 1.0497237, 1.0), (85, 59.999996, 0.0), (6, 59.999996, 1.0), 
     ]], 0, 380, True),
]:
    import MLE.MLE

    from pprint import pformat
    print(pformat(MLE.MLE.estimate(datagen=lambda: data, 
                                   wmin=wmin, wmax=wmax, rmin=0, rmax=1, raiseonerr=True, censored=censored)))
  
del data, wmin, wmax, censored

## Comparison with CVX

In [1]:
class DoubleDouble:
    def __init__(self, numactions, seed, wsupport, expwsq):
        import numpy
        import environments.ControlledRangeVariance
        
        self.numactions = numactions
        self.state = numpy.random.RandomState(seed+1)
        self.env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=seed, wsupport=wsupport, expwsq=expwsq)
        
    def range(self):
        return self.env.range()
        
    def rawsample(self, ndata):
        from collections import Counter
        (truevalue, data) = self.env.sample(ndata)
        
        nicedata = Counter()
        for c, w, r in data:
            actioncounts = Counter(self.state.choice(a=self.numactions, p=None, size=c))
            for pia, ca in actioncounts.items():
                nicedata.update({ (w, r, pia): ca })
            
        return (truevalue, [ (c, w, r, pia) for (w, r, pia), c in nicedata.items() ])
            
    def sample(self, ndata):
        from collections import Counter

        (truevalue, data) = self.rawsample(ndata)
        nicedata = Counter()
        for c, w, r, pia in data:
            nicedata.update({ (w, r): c })
        return (truevalue, [ (c, w, r) for (w, r), c in nicedata.items() ])
    
    def samplewithcvs(self, ndata):
        from collections import Counter
        import numpy

        (truevalue, data) = self.rawsample(ndata)
   
        nicedata = Counter()
    
        for c, w, r, pia in data:
            cvs = tuple(
                       w - 1 if a == pia else 0 for a in range(self.numactions)
            )
            nicedata.update({ (w, r, cvs): c})

        return (truevalue, [ (c, w, r, numpy.array(cv)) for (w, r, cv), c in nicedata.items() ])

# CVXPY (primal) implementation

class MLETest:
    @staticmethod
    def bitgen(minv, maxv):
        def bitgenhelp(vals, minv, maxv, pos, length):
            if pos >= length:
                yield tuple(vals)
            else:
                vals[pos] = minv[pos]
                yield from bitgenhelp(vals, minv, maxv, pos+1, length)
                vals[pos] = maxv[pos]
                yield from bitgenhelp(vals, minv, maxv, pos+1, length)
            
        assert len(minv) == len(maxv)
        length = len(minv)
        yield from bitgenhelp([None]*length, minv, maxv, 0, length) 

    @staticmethod
    def cvxdualestimate(data, wmin, wmax, cvmin, cvmax, rmin=0, rmax=1):
        import cvxpy as cp
        import numpy as np
        from collections import defaultdict
        
        cdict = defaultdict(int)
        n = 0
        for (ci, wi, ri, cvsi) in data:
            assert ci >= 0
            assert wi >= wmin and wi <= wmax
            assert ri >= rmin and ri <= rmax
            assert np.all(cvsi >= cvmin)
            assert np.all(cvsi <= cvmax)
            if ci > 0:
                cdict[(wi, ri, tuple(cvsi))] += ci
            n += ci
            ncvs = len(cvsi)
        assert n >= 1
        cdict.default_factory = None
        
        C = np.array([ value / n for key, value in cdict.items() ])
        wvec = np.array([ w for (w, _, _), _ in cdict.items() ])
        cvsmat = np.array([ cvs for (_, _, cvs), _ in cdict.items() ])

        gamma = cp.Variable()
        delta = cp.Variable(ncvs)
        
#         from pprint import pformat
#         print(pformat({
#             'cdict': cdict,
#             'C': C,
#             'wvec': wvec,
#             'cvsmat': cvsmat,
#             'gamma': gamma,
#             'delta': delta,
#         }))        
        
        prob = cp.Problem(cp.Maximize(cp.sum(cp.multiply(C, cp.log(n + gamma * (wvec - 1) + cp.matmul(cvsmat, delta))))), [
           n + gamma * (w - 1) + cp.matmul(np.array(bitvec), delta) >= 0
           for w in (wmin, wmax)
           for bitvec in MLETest.bitgen(cvmin, cvmax)
        ])
        prob.solve(solver='ECOS', max_iters=200)
        
        Q = np.array([ c / (n + gamma.value * (w - 1) + delta.value.dot(np.array(cvs)))
                       for (w, r, cvs), c in cdict.items() ])
        rawsumofw = wvec.dot(Q)
        rawsumofone = np.sum(Q)
        rawsumofcvs = np.matmul(cvsmat.T, Q)
        
        vhat = 0
        for (w, r, cvs), c in cdict.items():
            q = c / (n + gamma.value * (w - 1) + delta.value.dot(np.array(cvs)))
            vhat += q * w * r
           
        # NB: no explicit primal reconstruction necessary (!)
        vmin = vhat + max(0.0, 1 - rawsumofw) * rmin
        vmax = vhat + max(0.0, 1 - rawsumofw) * rmax
        vhat += max(0.0, 1 - rawsumofw) * (rmax - rmin) / 2
        
        from scipy.special import xlogy

        return vhat, {
            'vmin': vmin,
            'vmax': vmax,
            'gamma': gamma.value,
            'delta': delta.value,
            'qstar': { (w, r, cvs): c / (n + gamma.value * (w - 1) + delta.value.dot(np.array(cvs))) 
                       for (w, r, cvs), c in cdict.items() if c > 0  },
            'likelihood': np.sum(xlogy(C, Q)),
            'rawsumofone': rawsumofone,
            'rawsumofw': rawsumofw,
            'rawsumofcvs': rawsumofcvs,
        }
        
    @staticmethod
    def cvxestimate(data, wmin, wmax, cvmin, cvmax, rmin=0, rmax=1):
        import cvxpy as cp
        import numpy as np
        from collections import defaultdict
        
        cdict = defaultdict(int)
        n = 0
        for (ci, wi, ri, cvsi) in data:
            assert ci >= 0
            assert wi >= wmin and wi <= wmax
            assert ri >= rmin and ri <= rmax
            assert np.all(cvsi >= cvmin)
            assert np.all(cvsi <= cvmax)
            if ci > 0:
                cdict[(wi, ri, tuple(cvsi))] += ci
            n += ci
        assert n >= 1
        for bitvec in MLETest.bitgen(cvmin, cvmax):
            cdict[(wmin, rmin, bitvec)] += 0
            cdict[(wmin, rmax, bitvec)] += 0
            cdict[(wmax, rmin, bitvec)] += 0
            cdict[(wmax, rmax, bitvec)] += 0
        cdict.default_factory = None
                       
        C = np.array([ value / n for key, value in cdict.items() ])
        wvec = np.array([ w for (w, _, _), _ in cdict.items() ])
        cvsmat = np.array([ cvs for (_, _, cvs), _ in cdict.items() ])
        Q = cp.Variable(len(C))
                       
#         from pprint import pformat
#         print(pformat({
#             'cdict': cdict,
#             'C': C,
#             'wvec': wvec,
#             'cvsmat': cvsmat,
#             'Q': Q
#         }))
                       
        prob = cp.Problem(cp.Maximize(cp.sum(cp.multiply(C, cp.log(Q)))), [
            cp.sum(cp.multiply(wvec / wmax, Q)) == 1 / wmax,
            cp.sum(Q) == 1,
            cp.matmul(cvsmat.T, Q) == 0
        ])
        prob.solve(solver='ECOS')
        vmax, vmin = 0, 0
        for ((w, r, _), c), q in zip(cdict.items(), Q.value):
            if c > 0:
                vmax += q * w * r
                vmin += q * w * r
            else:
                vmax += q * w * rmax
                vmin += q * w * rmin
                                          
        vhat = (vmin + vmax) / 2              
 
        from scipy.special import xlogy
    
        return vhat, { 
            'vmax': vmax,
            'vmin': vmin,
            'qstar': { key: q for (key, c), q in zip(cdict.items(), Q.value) if c > 0 },
            'qex': { key: q for (key, c), q in zip(cdict.items(), Q.value) if c == 0 and q > 0},
            'likelihood': np.sum(xlogy(C, Q.value)),
            'sumofone': np.sum(Q.value),
            'sumofw': wvec.dot(Q.value),
            'sumofcvs': np.matmul(cvsmat.T, Q.value)
        }
    
def flass():
    from importlib import reload
    import numpy as np
    import MLE.MLE
    reload(MLE.MLE)
    
    ddm = DoubleDouble(numactions=2, seed=45, wsupport=[0, 2, 1000], expwsq=100)
    (truevalue, data) = ddm.samplewithcvs(20)
    return (truevalue, data, 
            MLETest.cvxestimate(data, wmin=0, wmax=1000, cvmin=-1*np.ones(2), cvmax=(1000-1)*np.ones(2)),
            MLETest.cvxdualestimate(data, wmin=0, wmax=1000, cvmin=-1*np.ones(2), cvmax=(1000-1)*np.ones(2)),
            MLE.MLE.estimatewithcv(lambda: data, wmin=0, wmax=1000, 
                                   cvmin=-1*np.ones(2), cvmax=(1000-1)*np.ones(2), raiseonerr=True)
           ) 

flass()

(0.7845508143539998,
 [(6, 0, 0, array([ 0, -1])),
  (5, 0, 0, array([-1,  0])),
  (6, 2, 1, array([1, 0])),
  (3, 2, 1, array([0, 1]))],
 (0.9252209718997872,
  {'vmax': 1.0000005451762404,
   'vmin': 0.850441398623334,
   'qstar': {(0, 0, (0, -1)): 0.29973625073879956,
    (0, 0, (-1, 0)): 0.274892585446737,
    (2, 1, (1, 0)): 0.27508014492234933,
    (2, 1, (0, 1)): 0.15014055438931767},
   'qex': {(0, 0, (-1.0, -1.0)): 1.3263744724756783e-07,
    (0, 1, (-1.0, -1.0)): 1.3263744724756783e-07,
    (1000, 0, (-1.0, -1.0)): 2.3403823948233652e-07,
    (1000, 1, (-1.0, -1.0)): 2.3403823948233652e-07,
    (0, 0, (-1.0, 999.0)): 3.2553409047166023e-07,
    (0, 1, (-1.0, 999.0)): 3.2553409047166023e-07,
    (1000, 0, (-1.0, 999.0)): 7.455561498388961e-05,
    (1000, 1, (-1.0, 999.0)): 7.455561498388961e-05},
   'likelihood': -1.3559347660273615,
   'sumofone': 0.9999999940468571,
   'sumofw': 1.0000005451762404,
   'sumofcvs': array([ 1.05740149e-09, -8.09757323e-10])}),
 (0.9252040916425

In [2]:
def testestimate():
    from importlib import reload
    from math import ceil
    import MLE.MLE
    import numpy as np
    from tqdm import tqdm as tqdm
    import sys

    reload(MLE.MLE)

    wsupport = [ 0, 2, 10 ]
    wmax = wsupport[-1]
    env = DoubleDouble(numactions=2, seed=45, wsupport=wsupport, expwsq=5)
    
    for ndata in tqdm(map(ceil, np.logspace(1, 7, 14)), file=sys.stderr, total=14):
        for i in range(101):
            (truevalue, data) = env.samplewithcvs(ndata)
            vhat, qstar = MLE.MLE.estimatewithcv(lambda: data, wmin=0, wmax=wmax, 
                                                 cvmin=-1*np.ones(2), cvmax=(wmax-1)*np.ones(2),
                                                 raiseonerr=True)
            try:
                cvxvhat, cvxqstar = MLETest.cvxdualestimate(data, wmin=0, wmax=wmax, 
                                                            cvmin=-1*np.ones(2), cvmax=(wmax-1)*np.ones(2))
            except:
                continue
 
            from pprint import pformat
            assert (   np.allclose(vhat, cvxvhat, atol=1e-3) 
                    or np.abs(vhat - truevalue) < np.abs(cvxvhat - truevalue)
                    or cvxqstar['rawsumofw'] > 1.001
                    or np.allclose(qstar['likelihood'], cvxqstar['likelihood'], atol=1e-4)
                   ), pformat(
            {
                'truevalue': truevalue,
                'data': [(c, w, r, cvs) for c, w, r, cvs in data if c > 0],
                'vhat': vhat,
                'cvxvhat': cvxvhat,
                'qstar': qstar,
                'cvxqstar': cvxqstar,
            })
                                    
testestimate()

100%|██████████| 14/14 [05:57<00:00, 87.01s/it]


# Comparison with baselines

In [ ]:
def produceresults(env, method, maxexp=5, numpts=20, ndataperpt=10000):
    from math import ceil
    import numpy as np
    
    wmin, wmax = env.range()

    for ndata in map(ceil, np.logspace(1, maxexp, numpts)):
        estimates=[]
        for i in range(1, ndataperpt+1):
            (truevalue, data) = env.sample(ndata)
            try:
                estimate = None
                estimate = method(data=data, wmin=wmin, wmax=wmax)
                assert np.isfinite(estimate)
            except:
                print('truevalue was {}'.format(truevalue))
                print('data was {}'.format(data))
                print('estimate was {}'.format(estimate))
                raise
            
            essden = sum(c*w*w for (c, w, _) in data)
            essnum = sum(c*w for (c, w, _) in data)
            ess = 0 if essden == 0 else essnum*(essnum/essden)
                                                
            estimates.append(
                ( truevalue,
                  truevalue - estimate,
                  (truevalue - estimate)**2,
                 ess
                )  
            )
            
        yield (ndata,
                { 
                    'bias': np.abs(sum(x[1] for x in estimates) / len(estimates)),
                    'mse': sum(x[2] for x in estimates) / len(estimates),
                    'ess': sum(x[3] for x in estimates) / len(estimates),
                },
              )
        
%matplotlib inline

class FlassPlot:
    @staticmethod
    def pic(x, y, label):
        import matplotlib.pyplot as plt
        
        plt.loglog(x, y, label=label)
        plt.legend()
        
    @staticmethod
    def forpaper():
        import matplotlib.pyplot as plt

        SMALL_SIZE = 10
        MEDIUM_SIZE = 16
        BIGGER_SIZE = 20

        plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
        plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
        plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
        plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
        plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
        
    @staticmethod
    def axeslabel(xlabel, ylabel):
        import matplotlib.pyplot as plt
        
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        
    @staticmethod
    def title(title):
        import matplotlib.pyplot as plt
        
        plt.title(title)
        
    @staticmethod
    def savefig(filename):
        import matplotlib.pyplot as plt
        
        plt.savefig(filename, bbox_inches='tight')
    
class ClippedDR:
    @staticmethod
    def estimate(data, baseline=0.5, **kwargs):
        import numpy as np
        n = sum(c for c, _, _ in data)
        return baseline if n == 0 else np.clip(sum(c*w*(r-baseline)+c*baseline for c, w, r in data) / n, a_min=0, a_max=1)
    
class SNIPS:
    @staticmethod
    def estimate(data, **kwargs):
        effn = sum(c*w for c, w, _ in data)
        return 0.5 if effn == 0 else sum(c*w*r for c, w, r in data) / effn

In [ ]:
from importlib import reload
import MLE.MLE

reload(MLE.MLE)
  
class DoubleDouble:
    def __init__(self, numactions, seed, wsupport, expwsq):
        import numpy
        import environments.ControlledRangeVariance
        
        self.numactions = numactions
        self.state = numpy.random.RandomState(seed+1)
        self.env = environments.ControlledRangeVariance.ControlledRangeVariance(seed=seed, wsupport=wsupport, expwsq=expwsq)
        
    def range(self):
        return self.env.range()
        
    def rawsample(self, ndata):
        from collections import Counter
        (truevalue, data) = self.env.sample(ndata)
        
        nicedata = Counter()
        for c, w, r in data:
            actioncounts = Counter(self.state.choice(a=self.numactions, p=None, size=c))
            for pia, ca in actioncounts.items():
                nicedata.update({ (w, r, pia): ca })
            
        return (truevalue, [ (c, w, r, pia) for (w, r, pia), c in nicedata.items() ])
            
    def sample(self, ndata):
        from collections import Counter

        (truevalue, data) = self.rawsample(ndata)
        nicedata = Counter()
        for c, w, r, pia in data:
            nicedata.update({ (w, r): c })
        return (truevalue, [ (c, w, r) for (w, r), c in nicedata.items() ])
    
    def samplewithcvs(self, ndata):
        from collections import Counter
        import numpy

        (truevalue, data) = self.rawsample(ndata)
   
        nicedata = Counter()
    
        for c, w, r, pia in data:
            cvs = tuple(
                       w - 1 if a == pia else 0 for a in range(self.numactions)
            )
            nicedata.update({ (w, r, cvs): c})

        return (truevalue, [ (c, w, r, numpy.array(cv)) for (w, r, cv), c in nicedata.items() ])

def getenv():
    ddm = DoubleDouble(numactions=2, seed=5, wsupport=[0, 2, 1000], expwsq=100)
    return (ddm, 1000)

from pprint import pformat
print(pformat(getenv()[0].rawsample(20)))
print(pformat(getenv()[0].sample(20)))
print(pformat(getenv()[0].samplewithcvs(20)))

FlassPlot.forpaper()
wmax = getenv()[1]
for (name, method) in [ ('Constant 0.5', lambda **kwargs: 0.5),
                        ('ClippedDR', ClippedDR.estimate),
                        ('SNIPS', SNIPS.estimate),
                        ('MLE', lambda data, **kwargs: MLE.MLE.estimate(datagen=lambda: data, **kwargs)[0]),
                      ]:
    print('****** {} ******'.format(name))
    res = []
    for zzz in produceresults(getenv()[0], method, maxexp=4, numpts=10, ndataperpt=1000):
        res.append(zzz)
        #print('{}'.format(zzz), flush=True)
    FlassPlot.pic([ x[0] / wmax for x in res], [ x[1]['mse'] for x in res], label=name)
FlassPlot.axeslabel('n / wmax', 'mse')
# FlassPlot.title('synthetic epsilon-greedy: estimation error')
#FlassPlot.savefig('epsilongreedy.mse.pdf')